### Imports

In [1]:
import os
import torch
# os.environ['TORCH'] = torch.__version__
print(torch.__version__)

# !pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
# !pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
# !pip install -q git+https://github.com/pyg-team/pytorch_geometric.git
# conda install pyg -c pyg
!nvcc --version

2.0.0
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0


In [2]:
import argparse, time

import dgl
import networkx as nx
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from dgi.dgi import Classifier, DGI
from dgl import DGLGraph
from dgl.data import load_data, register_data_args, DGLDataset
from torch_geometric.data import Data
import torch
import pandas as pd
import torch_geometric.transforms as T

import os

In [3]:
from tqdm import tqdm
from vgae.utils_vgae_tg import *
from utils.utils_go import *

import json
import pandas as pd

# os.environ["DGLBACKEND"] = "pytorch"

%load_ext autotime

time: 164 µs (started: 2023-10-28 10:30:17 -05:00)


In [4]:
# torch.manual_seed(42)
# np.random.seed(42)

time: 399 µs (started: 2023-10-28 10:30:17 -05:00)


### Parameters

In [5]:
file = open("exp.json")
experiment = json.load(file)
exp_num = experiment["exp"]

file = open("output/{}/parameters.json".format(exp_num))
params = json.load(file)

exp = params["exp"]
print("Exp:\t\t", exp)

method = "dgi"
print("Method:\t\t", method)

data_variations = params["data_variations"]
print("Data variations:", data_variations)

dimension = params["dimension"]
print("Dimension:\t", dimension)

groups_id = params["groups_id"]
print("Groups id:\t", groups_id)

subgroups_id = params["subgroups_id"]
print("Subgroups id:\t", subgroups_id)

Exp:		 exp12
Method:		 dgi
Data variations: ['none', 'str', 'dyn']
Dimension:	 3
Groups id:	 ['pck1', 'zwf1', 'WT']
Subgroups id:	 {'pck1': ['1', '2', '3'], 'zwf1': ['1', '2'], 'WT': ['1', '2', '3', '4', '5']}
time: 3.06 ms (started: 2023-10-28 10:30:17 -05:00)


### Node embeddings

In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

nodes_data = pd.read_csv("output/{}/preprocessing/graphs_data/nodes_data_{}_{}.csv".format(exp, groups_id[0], subgroups_id[groups_id[0]][0])).iloc[:, 2:]
edges_data = pd.read_csv("output/{}/preprocessing/graphs_data/edges_data_{}_{}.csv".format(exp, groups_id[0], subgroups_id[groups_id[0]][0]))

transform = T.Compose([
    # T.NormalizeFeatures(),
    T.ToDevice(device),
    # T.ToUndirected(),
    T.RandomLinkSplit(num_val=0.05, num_test=0.1, is_undirected=True, split_labels=True, add_negative_train_samples=False),
    # T.RandomNodeSplit(num_val=0.05, num_test=0.1),
])

dataset = CustomDataset(nodes_data, edges_data, transform=transform)
train_data, val_data, test_data = dataset[0]
dataset[0]

/home/ealvarez/miniconda3/envs/meta_net_3.10/lib/python3.10/site-packages/torch_geometric/transforms/random_link_split.py:220: UserWarning: There are not enough negative edges to satisfy the provided sampling ratio. The ratio will be adjusted to 0.39.
  warnings.warn(


(Data(x=[120, 24], edge_index=[2, 11796], pos_edge_label=[5898], pos_edge_label_index=[2, 5898]),
 Data(x=[120, 24], edge_index=[2, 11796], pos_edge_label=[346], pos_edge_label_index=[2, 346], neg_edge_label=[135], neg_edge_label_index=[2, 135]),
 Data(x=[120, 24], edge_index=[2, 12488], pos_edge_label=[693], pos_edge_label_index=[2, 693], neg_edge_label=[271], neg_edge_label_index=[2, 271]))

time: 228 ms (started: 2023-10-28 10:30:26 -05:00)


In [12]:
edges_data

,source,target,weight
0,0,1,-0.162508
1,0,2,-0.057338
2,0,3,0.083215
3,0,4,-0.562044
4,0,5,-0.026301
...,...,...,...
6932,116,118,-0.086813
6933,116,119,0.367687
6934,117,118,0.247453
6935,117,119,-0.278296


time: 8.48 ms (started: 2023-10-28 10:33:21 -05:00)


In [7]:
train_data.is_directed(), val_data.is_directed(), test_data.is_directed()

(False, False, False)

time: 46.5 ms (started: 2023-10-28 10:30:31 -05:00)


In [8]:
dataset.info()

Validate:	 True
Num. nodes:	 120
Num. edges:	 13874
Num. features:	 24
Has isolated:	 False
Has loops:	 False
Is directed:	 False
Is undirected:	 True
time: 15.4 ms (started: 2023-10-28 10:30:33 -05:00)


/home/ealvarez/miniconda3/envs/meta_net_3.10/lib/python3.10/site-packages/torch_geometric/data/in_memory_dataset.py:284: UserWarning: It is not recommended to directly access the internal storage format `data` of an 'InMemoryDataset'. If you are absolutely certain what you are doing, access the internal storage via `InMemoryDataset._data` instead to suppress this warning. Alternatively, you can access stacked individual attributes of every graph via `dataset.{attr_name}`.
  warnings.warn(msg)


In [9]:
# get node embeddings
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
seed = 42

for data_variation in data_variations:
    if data_variation != "none":
        for group in groups_id:
            subgroups_id[group] = [data_variation]
        torch.manual_seed(seed)
        np.random.seed(seed)
    else:
        torch.manual_seed(seed)
        np.random.seed(seed)

    print("Subgroups id:\t", subgroups_id)
    
    for group in tqdm(groups_id):
        for subgroup in tqdm(subgroups_id[group]):
            nodes_data = pd.read_csv("output/{}/preprocessing/graphs_data/nodes_data_{}_{}.csv".format(exp, group, subgroup)).iloc[:, 2:]
            edges_data = pd.read_csv("output/{}/preprocessing/graphs_data/edges_data_{}_{}.csv".format(exp, group, subgroup))

            # read dataset
            transform = T.Compose([
                T.NormalizeFeatures(),
                T.ToDevice(device),
                T.RandomLinkSplit(num_val=0.05, num_test=0.1, is_undirected=True, split_labels=True, add_negative_train_samples=False),
                # T.RandomNodeSplit(num_val=0.05, num_test=0.1),
            ])
            dataset = CustomDataset(nodes_data, edges_data, transform=transform)
            train_data, val_data, test_data = dataset[0]

            # train
            model = VGAE(Encoder(dataset.num_features, dimension)).to(device)
            train_vgae_tg(exp, model, train_data, test_data, method, group, subgroup, 0)
            

Subgroups id:	 {'pck1': ['1', '2', '3'], 'zwf1': ['1', '2'], 'WT': ['1', '2', '3', '4', '5']}


 67%|██████▋   | 2/3 [00:26<00:13, 13.14s/it]


KeyboardInterrupt: 

time: 26.8 s (started: 2023-10-26 15:25:48 -05:00)


In [ ]:
df_node_embeddings = pd.read_csv("output/{}/node_embeddings/node-embeddings_{}_{}_{}_{}.csv".format(exp, method, groups_id[0], 
                                                                                                    subgroups_id[groups_id[0]][0], 0), index_col=0)
df_node_embeddings

,0,1,2
0,0.000935,-0.013585,0.017064
1,-0.000733,-0.016786,0.016931
2,0.002580,-0.010430,0.017195
3,0.002752,-0.010103,0.017208
4,0.001849,-0.011833,0.017137
...,...,...,...
115,0.005171,-0.005463,0.017401
116,0.006135,-0.003615,0.017477
117,-0.002376,-0.019939,0.016800
118,0.001006,-0.013451,0.017069


time: 7.52 ms (started: 2023-10-23 15:00:20 -05:00)
